In [51]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import time
import re
from selenium import webdriver
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

# 검색하고 클릭하는 크롤러만들어보기 (사용X)

In [4]:
driver = webdriver.Chrome('C:/Users/rlath/Downloads/chromedriver')
url = 'https://www.nemoapp.kr/Search/MarketSalePrice?DealYear=2018&BuildingCategory=1&SWLng=126.95829981012457&SWLat=37.581020095242515&NELng=127.11983316583746&NELat=37.675372128021564&Zoom=13'
driver.get(url)
driver.maximize_window()
time.sleep(1)

html = driver.page_source
bs = BeautifulSoup(html, 'html5lib')

input_element = driver.find_element_by_id("keyword")
input_element.send_keys('서울특별시 동작구 상도동')
time.sleep(2)

click_element = driver.find_element_by_css_selector('#searchPlaceHolder > div > div.sg-market-search-box.sg-search-box-shadow > div.sg-top-search > form > div > span > button')
click_element.click()


# requests를 활용해 한 번에 해결~!

In [12]:
area_info = response.json()
area_info

{'regionCode': '11560132',
 'regionInfo': '서울특별시 영등포구 신길동',
 'submunicipality': {'code': '11560132',
  'name': '신길동',
  'center': {'latitude': 37.5060646891545, 'longitude': 126.913614508554},
  'municipality': {'code': '11560',
   'name': '영등포구',
   'province': {'code': '11', 'name': '서울특별시'}}},
 'zoom': 13,
 'prices': [{'dealYear': 2012, 'dealCount': 14, 'medianUnitPrice': 5000000},
  {'dealYear': 2013, 'dealCount': 15, 'medianUnitPrice': 6967966},
  {'dealYear': 2014, 'dealCount': 24, 'medianUnitPrice': 6274848},
  {'dealYear': 2015, 'dealCount': 42, 'medianUnitPrice': 5598482},
  {'dealYear': 2016, 'dealCount': 81, 'medianUnitPrice': 6802632},
  {'dealYear': 2017, 'dealCount': 51, 'medianUnitPrice': 8502674},
  {'dealYear': 2018, 'dealCount': 20, 'medianUnitPrice': 8709496}],
 'classes': [{'buildingClass': 1,
   'buildingClassName': '사무실',
   'dealCount': 1,
   'medianUnitPrice': 9136212},
  {'buildingClass': 2,
   'buildingClassName': '상가',
   'dealCount': 17,
   'medianUnitPrice'

In [118]:
seoul_areas = pd.read_csv('data/seoul_official_areas_with_code.csv')
seoul_areas.head()

,법정동명,법정동코드
0,서울특별시 강남구 개포동,1168010300
1,서울특별시 강남구 논현동,1168010800
2,서울특별시 강남구 대치동,1168010600
3,서울특별시 강남구 도곡동,1168011800
4,서울특별시 강남구 삼성동,1168010500


In [119]:
seoul_areas[seoul_areas['법정동명'].str.contains('동작동')]

,법정동명,법정동코드
84,서울특별시 동작구 동작동,1159010600


In [120]:
df_crawled_result = seoul_areas.copy(deep=True)
for year in range(2012, 2019):
    df_crawled_result[year] = np.nan
df_crawled_result.head()

,법정동명,법정동코드,2012,2013,2014,2015,2016,2017,2018
0,서울특별시 강남구 개포동,1168010300,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,서울특별시 강남구 논현동,1168010800,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,서울특별시 강남구 대치동,1168010600,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,서울특별시 강남구 도곡동,1168011800,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,서울특별시 강남구 삼성동,1168010500,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
for i in df_crawled_result.index:
    area_code = df_crawled_result.iloc[i]['법정동코드']
    url = f'https://www.nemoapp.kr/api/market-sale-prices/{area_code//100}?DealYear=2018&BuildingCategory=1'
    area_info = requests.get(url).json()
    for year in range(2012, 2019):
        price = area_info['prices'][year-2012]['medianUnitPrice']
        if price != 0.0:
            df_crawled_result[year].iloc[i] = price
df_crawled_result.head(20)

C:\Users\rlath\AppData\Roaming\Python\Python36\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,법정동명,법정동코드,2012,2013,2014,2015,2016,2017,2018
0,서울특별시 강남구 개포동,1168010300,9302014.0,8514523.0,7023023.0,9594692.0,11393230.0,11003020.0,12569340.0
1,서울특별시 강남구 논현동,1168010800,19782840.0,13613010.0,13813310.0,16146880.0,16461480.0,18294950.0,19686620.0
2,서울특별시 강남구 대치동,1168010600,17408480.0,12412720.0,17120620.0,16783510.0,18953840.0,22853580.0,23403270.0
3,서울특별시 강남구 도곡동,1168011800,9614812.0,10178120.0,11672860.0,12636740.0,14716570.0,17019420.0,16619180.0
4,서울특별시 강남구 삼성동,1168010500,17248460.0,14325310.0,16055990.0,19186850.0,17913590.0,27792000.0,26998410.0
5,서울특별시 강남구 세곡동,1168011100,NaN,NaN,NaN,NaN,NaN,11399510.0,NaN
6,서울특별시 강남구 수서동,1168011500,4881154.0,NaN,12264150.0,12905860.0,NaN,NaN,21503760.0
7,서울특별시 강남구 신사동,1168010700,24181990.0,19343700.0,20000000.0,20497800.0,21712910.0,23225120.0,25479930.0
8,서울특별시 강남구 압구정동,1168011000,NaN,38063872.0,48263392.0,NaN,NaN,NaN,35305340.0
9,서울특별시 강남구 역삼동,1168010100,13040380.0,15429580.0,14626820.0,17109300.0,18752420.0,18797260.0,23369690.0


In [133]:
df_crawled_result = df_crawled_result[['법정동코드', '법정동명', 2012, 2013, 2014, 2015, 2016, 2017, 2018]]
df_crawled_result.head()

,법정동코드,법정동명,2012,2013,2014,2015,2016,2017,2018
0,1168010300,서울특별시 강남구 개포동,9302014.0,8514523.0,7023023.0,9594692.0,11393230.0,11003020.0,12569340.0
1,1168010800,서울특별시 강남구 논현동,19782840.0,13613010.0,13813310.0,16146880.0,16461480.0,18294950.0,19686620.0
2,1168010600,서울특별시 강남구 대치동,17408480.0,12412720.0,17120620.0,16783510.0,18953840.0,22853580.0,23403270.0
3,1168011800,서울특별시 강남구 도곡동,9614812.0,10178120.0,11672860.0,12636740.0,14716570.0,17019420.0,16619180.0
4,1168010500,서울특별시 강남구 삼성동,17248460.0,14325310.0,16055990.0,19186850.0,17913590.0,27792000.0,26998410.0


In [134]:
df_crawled_result.to_csv('data/seoul_rental_price_crawled.csv', index=False)

In [135]:
df_2018 = df_crawled_result[['법정동코드', '법정동명', 2018]].copy(deep=True)
df_2018.head()

,법정동코드,법정동명,2018
0,1168010300,서울특별시 강남구 개포동,12569340.0
1,1168010800,서울특별시 강남구 논현동,19686620.0
2,1168010600,서울특별시 강남구 대치동,23403270.0
3,1168011800,서울특별시 강남구 도곡동,16619180.0
4,1168010500,서울특별시 강남구 삼성동,26998410.0


In [137]:
info_2018 = df_2018[2018].describe()
info_2018[4]

8242723.25

In [139]:
df_2018['2018_bin'] = pd.cut(df_2018[2018], [0, info_2018[4], info_2018[5], info_2018[6], info_2018[7]], labels=[4, 3, 2, 1])
df_2018.head(10)

,법정동코드,법정동명,2018,2018_bin
0,1168010300,서울특별시 강남구 개포동,12569340.0,2.0
1,1168010800,서울특별시 강남구 논현동,19686620.0,1.0
2,1168010600,서울특별시 강남구 대치동,23403270.0,1.0
3,1168011800,서울특별시 강남구 도곡동,16619180.0,2.0
4,1168010500,서울특별시 강남구 삼성동,26998410.0,1.0
5,1168011100,서울특별시 강남구 세곡동,NaN,NaN
6,1168011500,서울특별시 강남구 수서동,21503760.0,1.0
7,1168010700,서울특별시 강남구 신사동,25479930.0,1.0
8,1168011000,서울특별시 강남구 압구정동,35305340.0,1.0
9,1168010100,서울특별시 강남구 역삼동,23369690.0,1.0


In [140]:
df_2018.to_csv('data/seoul_rental_price_binning_2018.csv', index=False)

In [142]:
df_2018[df_2018['법정동명'] == '서울특별시 마포구 연남동']

,법정동코드,법정동명,2018,2018_bin
108,1144012400,서울특별시 마포구 연남동,18147260.0,1


In [144]:
df_binning = df_crawled_result.copy(deep=True)
df_binning.head()

,법정동코드,법정동명,2012,2013,2014,2015,2016,2017,2018
0,1168010300,서울특별시 강남구 개포동,9302014.0,8514523.0,7023023.0,9594692.0,11393230.0,11003020.0,12569340.0
1,1168010800,서울특별시 강남구 논현동,19782840.0,13613010.0,13813310.0,16146880.0,16461480.0,18294950.0,19686620.0
2,1168010600,서울특별시 강남구 대치동,17408480.0,12412720.0,17120620.0,16783510.0,18953840.0,22853580.0,23403270.0
3,1168011800,서울특별시 강남구 도곡동,9614812.0,10178120.0,11672860.0,12636740.0,14716570.0,17019420.0,16619180.0
4,1168010500,서울특별시 강남구 삼성동,17248460.0,14325310.0,16055990.0,19186850.0,17913590.0,27792000.0,26998410.0


In [145]:
for year in range(2012, 2019):
    values_info = df_binning[year].describe()
    df_binning[f'{year}_bin'] = pd.cut(df_binning[year], [0, values_info[4], values_info[5], values_info[6], values_info[7]], labels=[4, 3, 2, 1])
df_binning.head(10)

,법정동코드,법정동명,2012,2013,2014,2015,2016,2017,2018,2012_bin,2013_bin,2014_bin,2015_bin,2016_bin,2017_bin,2018_bin
0,1168010300,서울특별시 강남구 개포동,9302014.0,8514523.0,7023023.0,9594692.0,11393230.0,11003020.0,12569340.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0
1,1168010800,서울특별시 강남구 논현동,19782840.0,13613010.0,13813310.0,16146880.0,16461480.0,18294950.0,19686620.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1168010600,서울특별시 강남구 대치동,17408480.0,12412720.0,17120620.0,16783510.0,18953840.0,22853580.0,23403270.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1168011800,서울특별시 강남구 도곡동,9614812.0,10178120.0,11672860.0,12636740.0,14716570.0,17019420.0,16619180.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0
4,1168010500,서울특별시 강남구 삼성동,17248460.0,14325310.0,16055990.0,19186850.0,17913590.0,27792000.0,26998410.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1168011100,서울특별시 강남구 세곡동,NaN,NaN,NaN,NaN,NaN,11399510.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
6,1168011500,서울특별시 강남구 수서동,4881154.0,NaN,12264150.0,12905860.0,NaN,NaN,21503760.0,4.0,NaN,1.0,2.0,NaN,NaN,1.0
7,1168010700,서울특별시 강남구 신사동,24181990.0,19343700.0,20000000.0,20497800.0,21712910.0,23225120.0,25479930.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,1168011000,서울특별시 강남구 압구정동,NaN,38063872.0,48263392.0,NaN,NaN,NaN,35305340.0,NaN,1.0,1.0,NaN,NaN,NaN,1.0
9,1168010100,서울특별시 강남구 역삼동,13040380.0,15429580.0,14626820.0,17109300.0,18752420.0,18797260.0,23369690.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [148]:
df_binning = df_binning[['법정동코드', '법정동명', '2012_bin', '2013_bin', '2014_bin', '2015_bin', '2016_bin', '2017_bin', '2018_bin']]

In [149]:
df_binning.loc[df_binning['법정동명'] == '서울특별시 마포구 연남동']

,법정동코드,법정동명,2012_bin,2013_bin,2014_bin,2015_bin,2016_bin,2017_bin,2018_bin
108,1144012400,서울특별시 마포구 연남동,3,2,2,2,2,2,1


In [150]:
df_binning.loc[df_binning['법정동명'] == '서울특별시 마포구 상수동']

,법정동코드,법정동명,2012_bin,2013_bin,2014_bin,2015_bin,2016_bin,2017_bin,2018_bin
100,1144011500,서울특별시 마포구 상수동,2,2,1,2,1,1,1


In [151]:
df_binning.loc[df_binning['법정동명'] == '서울특별시 관악구 봉천동']

,법정동코드,법정동명,2012_bin,2013_bin,2014_bin,2015_bin,2016_bin,2017_bin,2018_bin
41,1162010100,서울특별시 관악구 봉천동,3,3,3,3,2,2,3


In [152]:
df_binning.loc[df_binning['법정동명'] == '서울특별시 종로구 익선동']

,법정동코드,법정동명,2012_bin,2013_bin,2014_bin,2015_bin,2016_bin,2017_bin,2018_bin
353,1111013300,서울특별시 종로구 익선동,2,1,NaN,3,2,2,2


In [153]:
df_binning.loc[df_binning['법정동명'] == '서울특별시 마포구 망원동']

,법정동코드,법정동명,2012_bin,2013_bin,2014_bin,2015_bin,2016_bin,2017_bin,2018_bin
99,1144012300,서울특별시 마포구 망원동,3,3,3,3,2,2,3


In [154]:
df_binning.loc[df_binning['법정동명'] == '서울특별시 동작구 상도동']

,법정동코드,법정동명,2012_bin,2013_bin,2014_bin,2015_bin,2016_bin,2017_bin,2018_bin
88,1159010200,서울특별시 동작구 상도동,4,3,3,3,3,3,3


In [155]:
df_binning.loc[df_binning['법정동명'] == '서울특별시 강동구 명일동']

,법정동코드,법정동명,2012_bin,2013_bin,2014_bin,2015_bin,2016_bin,2017_bin,2018_bin
18,1174010100,서울특별시 강동구 명일동,3,2,3,3,3,2,3


In [156]:
df_binning.loc[df_binning['법정동명'] == '서울특별시 노원구 상계동']

,법정동코드,법정동명,2012_bin,2013_bin,2014_bin,2015_bin,2016_bin,2017_bin,2018_bin
64,1135010500,서울특별시 노원구 상계동,4,4,4,4,4,4,4


In [159]:
df_binning.to_csv('data/seoul_rental_price_binning_total.csv', index=False)

In [160]:
df_crawled_result.head()

,법정동코드,법정동명,2012,2013,2014,2015,2016,2017,2018
0,1168010300,서울특별시 강남구 개포동,9302014.0,8514523.0,7023023.0,9594692.0,11393230.0,11003020.0,12569340.0
1,1168010800,서울특별시 강남구 논현동,19782840.0,13613010.0,13813310.0,16146880.0,16461480.0,18294950.0,19686620.0
2,1168010600,서울특별시 강남구 대치동,17408480.0,12412720.0,17120620.0,16783510.0,18953840.0,22853580.0,23403270.0
3,1168011800,서울특별시 강남구 도곡동,9614812.0,10178120.0,11672860.0,12636740.0,14716570.0,17019420.0,16619180.0
4,1168010500,서울특별시 강남구 삼성동,17248460.0,14325310.0,16055990.0,19186850.0,17913590.0,27792000.0,26998410.0


In [161]:
df = df_crawled_result.drop(['법정동코드', '법정동명'], axis=1)
df.head()

,2012,2013,2014,2015,2016,2017,2018
0,9302014.0,8514523.0,7023023.0,9594692.0,11393230.0,11003020.0,12569340.0
1,19782840.0,13613010.0,13813310.0,16146880.0,16461480.0,18294950.0,19686620.0
2,17408480.0,12412720.0,17120620.0,16783510.0,18953840.0,22853580.0,23403270.0
3,9614812.0,10178120.0,11672860.0,12636740.0,14716570.0,17019420.0,16619180.0
4,17248460.0,14325310.0,16055990.0,19186850.0,17913590.0,27792000.0,26998410.0


In [164]:
price_data = {}
for year in range(2012, 2018):
    price_data[f'{year}~{year+1}'] = (df[year+1].values - df[year].values) * 100 / df[year].values
price_changes = pd.DataFrame.from_dict(price_data)
price_changes.head()

,2012~2013,2013~2014,2014~2015,2015~2016,2016~2017,2017~2018
0,-8.465812,-17.517129,36.617693,18.745135,-3.424929,14.235364
1,-31.187787,1.471387,16.893634,1.948364,11.137941,7.606853
2,-28.697279,37.928029,-1.969029,12.931324,20.574934,2.405269
3,5.858752,14.685816,8.257445,16.458596,15.648008,-2.351667
4,-16.947310,12.081274,19.499638,-6.636108,55.144781,-2.855462


In [165]:
price_changes[['법정동코드', '법정동명']] = df_crawled_result[['법정동코드', '법정동명']].copy(deep=True)
price_changes.head()

,2012~2013,2013~2014,2014~2015,2015~2016,2016~2017,2017~2018,법정동코드,법정동명
0,-8.465812,-17.517129,36.617693,18.745135,-3.424929,14.235364,1168010300,서울특별시 강남구 개포동
1,-31.187787,1.471387,16.893634,1.948364,11.137941,7.606853,1168010800,서울특별시 강남구 논현동
2,-28.697279,37.928029,-1.969029,12.931324,20.574934,2.405269,1168010600,서울특별시 강남구 대치동
3,5.858752,14.685816,8.257445,16.458596,15.648008,-2.351667,1168011800,서울특별시 강남구 도곡동
4,-16.947310,12.081274,19.499638,-6.636108,55.144781,-2.855462,1168010500,서울특별시 강남구 삼성동


In [166]:
price_changes.columns

Index(['2012~2013', '2013~2014', '2014~2015', '2015~2016', '2016~2017',
       '2017~2018', '법정동코드', '법정동명'],
      dtype='object')

In [188]:
price_changes = price_changes[['법정동코드', '법정동명', '2012~2013', '2013~2014', '2014~2015', '2015~2016', '2016~2017', '2017~2018']]
price_changes.head()

,법정동코드,법정동명,2012~2013,2013~2014,2014~2015,2015~2016,2016~2017,2017~2018
0,1168010300,서울특별시 강남구 개포동,-8.465812,-17.517129,36.617693,18.745135,-3.424929,14.235364
1,1168010800,서울특별시 강남구 논현동,-31.187787,1.471387,16.893634,1.948364,11.137941,7.606853
2,1168010600,서울특별시 강남구 대치동,-28.697279,37.928029,-1.969029,12.931324,20.574934,2.405269
3,1168011800,서울특별시 강남구 도곡동,5.858752,14.685816,8.257445,16.458596,15.648008,-2.351667
4,1168010500,서울특별시 강남구 삼성동,-16.947310,12.081274,19.499638,-6.636108,55.144781,-2.855462


In [189]:
price_changes.to_csv('data/seoul_rental_price_changes.csv', index=True)

In [190]:
for year in range(2012, 2018):
    values_info = price_changes[f'{year}~{year+1}'].describe()
    price_changes[f'{year}~{year+1}_bin'] = pd.cut(price_changes[f'{year}~{year+1}'], [values_info[3], values_info[4], values_info[5], values_info[6], values_info[7]], labels=[4, 3, 2, 1])
price_changes.head(10)

,법정동코드,법정동명,2012~2013,2013~2014,2014~2015,2015~2016,2016~2017,2017~2018,2012~2013_bin,2013~2014_bin,2014~2015_bin,2015~2016_bin,2016~2017_bin,2017~2018_bin
0,1168010300,서울특별시 강남구 개포동,-8.465812,-17.517129,36.617693,18.745135,-3.424929,14.235364,3.0,4.0,1.0,2.0,3.0,2.0
1,1168010800,서울특별시 강남구 논현동,-31.187787,1.471387,16.893634,1.948364,11.137941,7.606853,4.0,3.0,2.0,3.0,2.0,3.0
2,1168010600,서울특별시 강남구 대치동,-28.697279,37.928029,-1.969029,12.931324,20.574934,2.405269,4.0,1.0,3.0,2.0,2.0,3.0
3,1168011800,서울특별시 강남구 도곡동,5.858752,14.685816,8.257445,16.458596,15.648008,-2.351667,2.0,2.0,2.0,2.0,2.0,3.0
4,1168010500,서울특별시 강남구 삼성동,-16.947310,12.081274,19.499638,-6.636108,55.144781,-2.855462,3.0,2.0,2.0,3.0,1.0,3.0
5,1168011100,서울특별시 강남구 세곡동,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1168011500,서울특별시 강남구 수서동,NaN,NaN,5.232405,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
7,1168010700,서울특별시 강남구 신사동,-20.007824,3.392836,2.489000,5.928002,6.964566,9.708497,3.0,2.0,3.0,3.0,3.0,3.0
8,1168011000,서울특별시 강남구 압구정동,NaN,26.795803,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
9,1168010100,서울특별시 강남구 역삼동,18.321552,-5.202734,16.972110,9.603666,0.239116,24.324981,2.0,3.0,2.0,2.0,3.0,2.0


In [191]:
price_changes.columns

Index(['법정동코드', '법정동명', '2012~2013', '2013~2014', '2014~2015', '2015~2016',
       '2016~2017', '2017~2018', '2012~2013_bin', '2013~2014_bin',
       '2014~2015_bin', '2015~2016_bin', '2016~2017_bin', '2017~2018_bin'],
      dtype='object')

In [192]:
price_changes_binning = price_changes[['법정동코드', '법정동명', '2012~2013_bin', '2013~2014_bin', '2014~2015_bin', '2015~2016_bin', '2016~2017_bin', '2017~2018_bin']].copy()

In [193]:
price_changes_binning.head()

,법정동코드,법정동명,2012~2013_bin,2013~2014_bin,2014~2015_bin,2015~2016_bin,2016~2017_bin,2017~2018_bin
0,1168010300,서울특별시 강남구 개포동,3,4,1,2,3,2
1,1168010800,서울특별시 강남구 논현동,4,3,2,3,2,3
2,1168010600,서울특별시 강남구 대치동,4,1,3,2,2,3
3,1168011800,서울특별시 강남구 도곡동,2,2,2,2,2,3
4,1168010500,서울특별시 강남구 삼성동,3,2,2,3,1,3


In [181]:
price_changes_binning[price_changes_binning['법정동명'] == '서울특별시 마포구 연남동']

,법정동코드,법정동명,2012~2013_bin,2013~2014_bin,2014~2015_bin,2015~2016_bin,2016~2017_bin,2017~2018_bin
108,1144012400,서울특별시 마포구 연남동,1,2,2,2,3,1


In [182]:
price_changes_binning[price_changes_binning['법정동명'] == '서울특별시 마포구 상수동']

,법정동코드,법정동명,2012~2013_bin,2013~2014_bin,2014~2015_bin,2015~2016_bin,2016~2017_bin,2017~2018_bin
100,1144011500,서울특별시 마포구 상수동,4,1,4,1,2,2


In [183]:
price_changes_binning[price_changes_binning['법정동명'] == '서울특별시 마포구 망원동']

,법정동코드,법정동명,2012~2013_bin,2013~2014_bin,2014~2015_bin,2015~2016_bin,2016~2017_bin,2017~2018_bin
99,1144012300,서울특별시 마포구 망원동,2,3,2,2,1,4


In [194]:
price_changes_binning[price_changes_binning['법정동명'] == '서울특별시 종로구 익선동']

,법정동코드,법정동명,2012~2013_bin,2013~2014_bin,2014~2015_bin,2015~2016_bin,2016~2017_bin,2017~2018_bin
353,1111013300,서울특별시 종로구 익선동,1,NaN,NaN,1,2,2


In [195]:
price_changes_binning[price_changes_binning['법정동명'] == '서울특별시 관악구 봉천동']

,법정동코드,법정동명,2012~2013_bin,2013~2014_bin,2014~2015_bin,2015~2016_bin,2016~2017_bin,2017~2018_bin
41,1162010100,서울특별시 관악구 봉천동,3,2,3,2,2,3


In [187]:
price_changes_binning.to_csv('data/seoul_rental_price_changes_binning_total.csv', index=True)

In [196]:
price_changes_binning[price_changes_binning['법정동명'] == '서울특별시 동작구 상도동']

,법정동코드,법정동명,2012~2013_bin,2013~2014_bin,2014~2015_bin,2015~2016_bin,2016~2017_bin,2017~2018_bin
88,1159010200,서울특별시 동작구 상도동,1,3,2,3,2,4


In [197]:
price_changes_binning[price_changes_binning['법정동명'] == '서울특별시 노원구 상계동']

,법정동코드,법정동명,2012~2013_bin,2013~2014_bin,2014~2015_bin,2015~2016_bin,2016~2017_bin,2017~2018_bin
64,1135010500,서울특별시 노원구 상계동,2,3,3,2,2,3


In [198]:
price_changes_binning[price_changes_binning['법정동명'] == '서울특별시 강동구 명일동']

,법정동코드,법정동명,2012~2013_bin,2013~2014_bin,2014~2015_bin,2015~2016_bin,2016~2017_bin,2017~2018_bin
18,1174010100,서울특별시 강동구 명일동,1,4,2,2,1,4
